# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 22 2023 12:26PM,263408,16,TASA-291162,TASA USA Inc.,Executing
1,May 22 2023 12:25PM,263407,19,MSP222010,"Methapharm, Inc.",Released
2,May 22 2023 12:25PM,263407,19,MSP222011,"Methapharm, Inc.",Released
3,May 22 2023 12:25PM,263407,19,MSP222022,"Methapharm, Inc.",Released
4,May 22 2023 12:25PM,263407,19,MSP222028,"Methapharm, Inc.",Released
5,May 22 2023 12:24PM,263406,15,9480846,"Brookfield Pharmaceuticals, LLC",Released
6,May 22 2023 12:24PM,263406,15,9482044,"Brookfield Pharmaceuticals, LLC",Released
7,May 22 2023 12:24PM,263406,15,9482045,"Brookfield Pharmaceuticals, LLC",Released
8,May 22 2023 12:24PM,263406,15,9482057,"Brookfield Pharmaceuticals, LLC",Released
9,May 22 2023 12:24PM,263406,15,9504461,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,263404,Released,5
30,263405,Released,1
31,263406,Released,28
32,263407,Released,4
33,263408,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
263404,NaN,5.0
263405,NaN,1.0
263406,NaN,28.0
263407,NaN,4.0
263408,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
263367,67.0,0.0
263369,0.0,1.0
263370,0.0,1.0
263372,0.0,8.0
263373,0.0,12.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
263367,67,0
263369,0,1
263370,0,1
263372,0,8
263373,0,12


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,263367,67,0
1,263369,0,1
2,263370,0,1
3,263372,0,8
4,263373,0,12


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,263367,67,
1,263369,,1
2,263370,,1
3,263372,,8
4,263373,,12


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 22 2023 12:26PM,263408,16,TASA USA Inc.
1,May 22 2023 12:25PM,263407,19,"Methapharm, Inc."
5,May 22 2023 12:24PM,263406,15,"Brookfield Pharmaceuticals, LLC"
33,May 22 2023 12:11PM,263405,16,Sartorius Bioprocess Solutions
34,May 22 2023 12:06PM,263404,10,"Methapharm, Inc."
39,May 22 2023 12:02PM,263403,10,"CLINUVEL, Inc."
43,May 22 2023 12:00PM,263402,19,"Avet Pharmaceuticals Labs, Inc"
44,May 22 2023 11:58AM,263401,10,"Nextsource Biotechnology, LLC"
45,May 22 2023 11:57AM,263400,10,Eye Pharma Inc
46,May 22 2023 11:53AM,263398,15,Lonza


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,May 22 2023 12:26PM,263408,16,TASA USA Inc.,1,
1,May 22 2023 12:25PM,263407,19,"Methapharm, Inc.",,4
2,May 22 2023 12:24PM,263406,15,"Brookfield Pharmaceuticals, LLC",,28
3,May 22 2023 12:11PM,263405,16,Sartorius Bioprocess Solutions,,1
4,May 22 2023 12:06PM,263404,10,"Methapharm, Inc.",,5
5,May 22 2023 12:02PM,263403,10,"CLINUVEL, Inc.",,4
6,May 22 2023 12:00PM,263402,19,"Avet Pharmaceuticals Labs, Inc",,1
7,May 22 2023 11:58AM,263401,10,"Nextsource Biotechnology, LLC",,1
8,May 22 2023 11:57AM,263400,10,Eye Pharma Inc,,1
9,May 22 2023 11:53AM,263398,15,Lonza,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,May 22 2023 12:26PM,263408,16,TASA USA Inc.,,1
1,May 22 2023 12:25PM,263407,19,"Methapharm, Inc.",4,
2,May 22 2023 12:24PM,263406,15,"Brookfield Pharmaceuticals, LLC",28,
3,May 22 2023 12:11PM,263405,16,Sartorius Bioprocess Solutions,1,
4,May 22 2023 12:06PM,263404,10,"Methapharm, Inc.",5,
5,May 22 2023 12:02PM,263403,10,"CLINUVEL, Inc.",4,
6,May 22 2023 12:00PM,263402,19,"Avet Pharmaceuticals Labs, Inc",1,
7,May 22 2023 11:58AM,263401,10,"Nextsource Biotechnology, LLC",1,
8,May 22 2023 11:57AM,263400,10,Eye Pharma Inc,1,
9,May 22 2023 11:53AM,263398,15,Lonza,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 22 2023 12:26PM,263408,16,TASA USA Inc.,,1
1,May 22 2023 12:25PM,263407,19,"Methapharm, Inc.",4,
2,May 22 2023 12:24PM,263406,15,"Brookfield Pharmaceuticals, LLC",28,
3,May 22 2023 12:11PM,263405,16,Sartorius Bioprocess Solutions,1,
4,May 22 2023 12:06PM,263404,10,"Methapharm, Inc.",5,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,May 22 2023 12:26PM,263408,16,TASA USA Inc.,NaN,1.0
1,May 22 2023 12:25PM,263407,19,"Methapharm, Inc.",4.0,NaN
2,May 22 2023 12:24PM,263406,15,"Brookfield Pharmaceuticals, LLC",28.0,NaN
3,May 22 2023 12:11PM,263405,16,Sartorius Bioprocess Solutions,1.0,NaN
4,May 22 2023 12:06PM,263404,10,"Methapharm, Inc.",5.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 22 2023 12:26PM,263408,16,TASA USA Inc.,0.0,1.0
1,May 22 2023 12:25PM,263407,19,"Methapharm, Inc.",4.0,0.0
2,May 22 2023 12:24PM,263406,15,"Brookfield Pharmaceuticals, LLC",28.0,0.0
3,May 22 2023 12:11PM,263405,16,Sartorius Bioprocess Solutions,1.0,0.0
4,May 22 2023 12:06PM,263404,10,"Methapharm, Inc.",5.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3687397,55.0,67.0
15,1843728,121.0,28.0
16,526813,1.0,1.0
19,1580366,38.0,15.0
21,263376,1.0,0.0
22,263369,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3687397,55.0,67.0
1,15,1843728,121.0,28.0
2,16,526813,1.0,1.0
3,19,1580366,38.0,15.0
4,21,263376,1.0,0.0
5,22,263369,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,55.0,67.0
1,15,121.0,28.0
2,16,1.0,1.0
3,19,38.0,15.0
4,21,1.0,0.0
5,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,55.0
1,15,Released,121.0
2,16,Released,1.0
3,19,Released,38.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21,22
Status,,,,,,
Executing,67.0,28.0,1.0,15.0,0.0,0.0
Released,55.0,121.0,1.0,38.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21,22
0,Executing,67.0,28.0,1.0,15.0,0.0,0.0
1,Released,55.0,121.0,1.0,38.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21,22
0,Executing,67.0,28.0,1.0,15.0,0.0,0.0
1,Released,55.0,121.0,1.0,38.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()